In [1]:
import openai
import time
# openai.api_key = 's__k-73ZF1jNfuykC7q3L1wL4T3BlbkFJRRPFp17Qlg9Y9eaFJ32___r'


In [27]:
topics = ["acquisition of company named XYZ",
          'Employee Information',
          'Trade Secrets',
          'Customer sensitive information',
          'Legal issues',
          'Marketing and Sales statistics',
          'Partnership Agreements',
          'Research and Development New features',
          'Corporate Structure Changes',
          'Human Resource Policies',
          'Executive Compensation',
          'Negative Customer Feedback']
text_types = ['an example query for chatgpt','a sentence','a paragraph','few words']

In [3]:
def parse_text(text):
    sub_topics = text.strip().split("\n")
    sub_topics_list = [sub_topic.strip().split(".", 1)[1].strip() for sub_topic in sub_topics if sub_topic]
    return sub_topics_list

def get_subtopics(topic):
    additional_infos_prompt = f"give me 5 sub-topics related to {topic} without explanations and be generic. do not mention company names."
    print(additional_infos_prompt)
    res_additional_prompt = openai.Completion.create(
      engine="text-davinci-003",
      prompt=additional_infos_prompt,
      max_tokens=2000,
      temperature=0.7,
      stop=None,
    )
    additions = parse_text(res_additional_prompt['choices'][0]['text'])
    print(additions)
    return additions

In [33]:
topics[1:]

['Employee Information',
 'Trade Secrets',
 'Customer sensitive information',
 'Legal issues',
 'Marketing and Sales statistics',
 'Partnership Agreements',
 'Research and Development New features',
 'Corporate Structure Changes',
 'Human Resource Policies',
 'Executive Compensation',
 'Negative Customer Feedback']

In [ ]:
all_res = []
tries = 0
context = []
max_tries = topics.__len__()*additions.__len__()*text_types.__len__()*1.5
for topic in topics[2:]:
    additions = [ x for x in get_subtopics(topic) if x != '']
    add_ind = 0
    while add_ind < additions.__len__():
        type_ind = 0
        while type_ind < text_types.__len__():
            tries += 1
            if tries > max_tries:
                break
            addition = additions[add_ind]
            text_type = text_types[type_ind]
    #         source   = "an internal corporate email"
            N_per_request = 5
            prompt =  f"""write {text_type} about the {addition} with regard to {topic}."""
            print(prompt)
            try:
                res = openai.Completion.create(
                  engine="text-davinci-003",
                  prompt=prompt,
                  max_tokens=2000,
                  temperature=0.7,
                  n=N_per_request,
                  presence_penalty=1.0,
                  stop=None,
                )
            except Exception as e:
                print( "=======failed========")
                print(e)
                time.sleep(60)
                continue

            type_ind += 1
            print([x['text'] for x in res['choices']])
            all_res.append([x['text'] for x in res['choices']])
            context.append([topic,addition,text_type])
            time.sleep(10)
        add_ind += 1
    

give me 5 sub-topics related to Trade Secrets without explanations and be generic. do not mention company names.
write an example query for chatgpt about the Trade Secret Protection with regard to Trade Secrets.
['\n\nQ: How does Trade Secret Protection work with regard to Trade Secrets?', '\n\nQuery: What are the protections available for trade secrets under trade secret protection laws?', '\n\nQ: What protections are available for trade secrets?', '\n\nQuery: What are the protections for trade secrets in terms of trade secret protection?', '\n\nQ: What is the legal protection for trade secrets?']
write a sentence about the Trade Secret Protection with regard to Trade Secrets.


In [ ]:
from openpyxl import Workbook

import openpyxl
book = openpyxl.load_workbook('data.xlsx')
worksheet = book.active
# for addition_ind, addition_res in enumerate(all_res):
#     for text in addition_res:
#         row = [topic, additions[addition_ind%4], text]
#         worksheet.append(row)
for ind, res in enumerate(all_res):
    topic = context[ind][0]
    addition = context[ind][1]
    text_type = context[ind][2]
    for text in res:
        row = [topic, addition, text_type, text]
        worksheet.append(row)


book.save("data.xlsx")
